<h1> Merging 3 Datasets

In [6]:
!pip install matplotlib seaborn scikit-learn

104.76s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 23.2 MB/s eta 0:00:00 0:00:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 25.0 MB/s eta 0:00:00a 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)


<h3>1.Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

<h3>2.Load data set

In [3]:
df_matches = pd.read_csv('result_clean_data.csv')
df_team_stats = pd.read_csv('squad_stats.csv')
df_opponent_stats = pd.read_csv('squad_stats_opponent.csv')

<h4> Display the basic info of the data set

In [4]:
print("Matches dataset shape:", df_matches.shape)
print("Team stats dataset shape:", df_team_stats.shape)
print("Opponent stats dataset shape:", df_opponent_stats.shape)

Matches dataset shape: (184, 18)
Team stats dataset shape: (36, 32)
Opponent stats dataset shape: (36, 32)


In [20]:
df_matches.isnull().sum().sum() #checking for missing values

np.int64(0)

In [19]:
df_matches.isnull().sum()

Round         0
Day           0
Date          0
Time          0
Home          0
xG            0
Score         0
xG.1          0
Away          0
Attendance    0
Venue         0
Referee       0
HomeGoals     0
AwayGoals     0
Outcome       0
dtype: int64

In [8]:
df_matches.head()

,Round,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes,HomeGoals,AwayGoals,Outcome
0,League phase,1.0,Tue,2024-09-17,18:45,Young Boys ch,0.6,0–3,2.0,eng Aston Villa,31500.0,Stadion Wankdorf,Georgi Kabakov,Match Report,NaN,0,3,-1
1,League phase,1.0,Tue,2024-09-17,18:45,Juventus it,2.9,3–1,1.0,nl PSV Eindhoven,40417.0,Allianz Stadium,Alejandro Hernández,Match Report,NaN,3,1,1
2,League phase,1.0,Tue,2024-09-17,20:00,Sporting CP pt,1.6,2–0,0.4,fr Lille,40024.0,Estádio José Alvalade,Donatas Rumšas,Match Report,NaN,2,0,1
3,League phase,1.0,Tue,2024-09-17,21:00,Real Madrid es,2.6,3–1,1.9,de Stuttgart,71288.0,Estadio Santiago Bernabéu,Halil Umut Meler,Match Report,NaN,3,1,1
4,League phase,1.0,Tue,2024-09-17,21:00,Milan it,0.6,1–3,3.1,eng Liverpool,59826.0,Stadio Giuseppe Meazza,Espen Eskås,Match Report,NaN,1,3,-1


In [10]:
df_matches = df_matches.drop('Wk', axis=1)
df_matches = df_matches.drop('Notes', axis=1)

In [15]:
df_matches.head()

,Round,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,HomeGoals,AwayGoals,Outcome
0,League phase,Tue,2024-09-17,18:45,Young Boys ch,0.6,0–3,2.0,eng Aston Villa,31500.0,Stadion Wankdorf,Georgi Kabakov,0,3,-1
1,League phase,Tue,2024-09-17,18:45,Juventus it,2.9,3–1,1.0,nl PSV Eindhoven,40417.0,Allianz Stadium,Alejandro Hernández,3,1,1
2,League phase,Tue,2024-09-17,20:00,Sporting CP pt,1.6,2–0,0.4,fr Lille,40024.0,Estádio José Alvalade,Donatas Rumšas,2,0,1
3,League phase,Tue,2024-09-17,21:00,Real Madrid es,2.6,3–1,1.9,de Stuttgart,71288.0,Estadio Santiago Bernabéu,Halil Umut Meler,3,1,1
4,League phase,Tue,2024-09-17,21:00,Milan it,0.6,1–3,3.1,eng Liverpool,59826.0,Stadio Giuseppe Meazza,Espen Eskås,1,3,-1


In [14]:
df_matches = df_matches.drop('Match Report',axis=1)

In [18]:
#To replace the 5 missing attendance value
# Group by venue and calculate median attendance
venue_median_attendance = df_matches.groupby('Venue')['Attendance'].transform('median')
# Fill missing values with venue-specific median
df_matches['Attendance'].fillna(venue_median_attendance, inplace=True)

/var/folders/3h/skhr58nn5kzfg7t04wby785c0000gn/T/ipykernel_44213/3154934565.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_matches['Attendance'].fillna(venue_median_attendance, inplace=True)


In [33]:
def clean_team_name(name):
    # Handle missing values
    if pd.isna(name):
        return name

    # List of country codes
    country_codes = ['eng', 'es', 'de', 'fr', 'it', 'pt', 'nl', 'be',
                     'at', 'ch', 'hr', 'cz', 'rs', 'ua', 'sk', 'sct']

    # Split the name into words
    parts = name.split()

    # Check if first word is a country code (e.g., "eng Arsenal")
    if len(parts) > 1 and parts[0].lower() in country_codes:
        return ' '.join(parts[1:])

    # Check if last word is a country code (e.g., "Juventus it")
    if len(parts) > 1 and parts[-1].lower() in country_codes:
        return ' '.join(parts[:-1])

    # Return original name if no country code found
    return name

In [34]:
df_matches['Home_clean'] = df_matches['Home'].apply(clean_team_name)
df_matches['Away_clean'] = df_matches['Away'].apply(clean_team_name)

In [35]:
df_matches.head()

,Round,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,HomeGoals,AwayGoals,Outcome,Home_clean,Away_clean
0,League phase,Tue,2024-09-17,18:45,Young Boys ch,0.6,0–3,2.0,eng Aston Villa,31500.0,Stadion Wankdorf,Georgi Kabakov,0,3,-1,Young Boys,Aston Villa
1,League phase,Tue,2024-09-17,18:45,Juventus it,2.9,3–1,1.0,nl PSV Eindhoven,40417.0,Allianz Stadium,Alejandro Hernández,3,1,1,Juventus,PSV Eindhoven
2,League phase,Tue,2024-09-17,20:00,Sporting CP pt,1.6,2–0,0.4,fr Lille,40024.0,Estádio José Alvalade,Donatas Rumšas,2,0,1,Sporting CP,Lille
3,League phase,Tue,2024-09-17,21:00,Real Madrid es,2.6,3–1,1.9,de Stuttgart,71288.0,Estadio Santiago Bernabéu,Halil Umut Meler,3,1,1,Real Madrid,Stuttgart
4,League phase,Tue,2024-09-17,21:00,Milan it,0.6,1–3,3.1,eng Liverpool,59826.0,Stadio Giuseppe Meazza,Espen Eskås,1,3,-1,Milan,Liverpool


In [36]:
# Extract team names from squad_stats
team_names = df_team_stats['Squad'].str.extract(r'([a-z]{2,3}\s)?(.*)')
df_team_stats['country_code'] = team_names[0].str.strip()
df_team_stats['team_name'] = team_names[1].str.strip()

In [37]:
df_team_stats.head()

,Squad,#Pl,Age,Poss,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,country_code,team_name
0,eng Arsenal,24,26.6,50.8,12,132,1080,12.0,29,18,...,3.92,2.25,3.75,1.88,1.22,3.09,1.55,2.77,eng,Arsenal
1,eng Aston Villa,25,27.8,47.3,12,132,1080,12.0,22,18,...,3.33,1.75,3.25,1.57,1.24,2.81,1.44,2.68,eng,Aston Villa
2,it Atalanta,24,28.4,53.3,10,110,900,10.0,22,19,...,4.10,2.10,4.00,2.45,1.89,4.34,2.20,4.09,it,Atalanta
3,es Atlético Madrid,23,29.2,47.4,10,110,930,10.3,22,17,...,3.77,2.13,3.77,1.41,1.17,2.58,1.41,2.58,es,Atlético Madrid
4,es Barcelona,26,25.8,59.8,12,132,1080,12.0,35,27,...,5.17,2.67,4.92,2.34,1.75,4.08,2.14,3.89,es,Barcelona


In [38]:
df_matches['Home_clean'].unique()

array(['Young Boys', 'Juventus', 'Sporting CP', 'Real Madrid', 'Milan',
       'Bayern Munich', 'Sparta Prague', 'Bologna', 'Celtic',
       'Manchester City', 'Club Brugge', 'Paris S-G', 'Red Star',
       'Feyenoord', 'Monaco', 'Atalanta', 'Brest', 'Atlético Madrid',
       'Stuttgart', 'RB Salzburg', 'Arsenal', 'Barcelona', 'Dortmund',
       'Inter', 'Slovan Bratislava', 'PSV Eindhoven', 'Leverkusen',
       'Girona', 'Shakhtar', 'Liverpool', 'Aston Villa', 'Benfica',
       'Dinamo Zagreb', 'RB Leipzig', 'Sturm Graz', 'Lille'], dtype=object)

In [39]:
df_team_stats['team_name'].unique()

array(['Arsenal', 'Aston Villa', 'Atalanta', 'Atlético Madrid',
       'Barcelona', 'Bayern Munich', 'Benfica', 'Bologna', 'Brest',
       'Celtic', 'Club Brugge', 'Dinamo Zagreb', 'Dortmund', 'Feyenoord',
       'Girona', 'Inter', 'Juventus', 'Leverkusen', 'Lille', 'Liverpool',
       'Manchester City', 'Milan', 'Monaco', 'Paris S-G', 'PSV Eindhoven',
       'RB Leipzig', 'RB Salzburg', 'Real Madrid', 'Red Star', 'Shakhtar',
       'Slovan Bratislava', 'Sparta Prague', 'Sporting CP', 'Sturm Graz',
       'Stuttgart', 'Young Boys'], dtype=object)

In [52]:
team_names = df_opponent_stats['Squad'].str.extract(r'([a-z]{2,3}\s)?(vs\s)?(.*)')
df_opponent_stats['country_code'] = team_names[0].str.strip() if team_names[0] is not None else None
df_opponent_stats['team_name'] = team_names[2].str.strip() if team_names[2] is not None else None

In [53]:
df_opponent_stats.head()

,Squad,#Pl,Age,Poss,Playing Time_MP,Playing Time_Starts,Playing Time_Min,Playing Time_90s,Performance_Gls,Performance_Ast,...,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG,country_code,team_name
0,eng vs Arsenal,24,26.7,49.2,12,132,1080,12.0,7,4,...,0.92,0.42,0.75,0.89,0.54,1.43,0.68,1.23,eng,Arsenal
1,eng vs Aston Villa,25,26.1,52.8,12,132,1080,12.0,12,8,...,1.67,0.92,1.58,1.16,0.87,2.03,1.10,1.97,eng,Aston Villa
2,it vs Atalanta,24,26.1,46.7,10,110,900,10.0,11,8,...,1.90,1.00,1.80,1.09,0.87,1.96,1.01,1.88,it,Atalanta
3,es vs Atlético Madrid,23,26.6,52.6,10,110,930,10.3,14,9,...,2.23,0.97,1.84,1.25,0.70,1.96,0.87,1.57,es,Atlético Madrid
4,es vs Barcelona,26,26.9,40.3,12,132,1080,12.0,16,10,...,2.17,1.08,1.92,1.51,1.00,2.51,1.31,2.31,es,Barcelona


<h4>Merging Home Team Stats

In [ ]:
df_merged = df_matches.merge(
    df_team_stats,
    left_on='Home_clean',  # Match results home team
    right_on='Squad_clean',  # Team stats squad name
    how='left',  # Keep all matches even if stats missing
    suffixes=('', '_home')  # Add _home suffix to avoid column name conflicts
)